Après avoir exploré et nettoyé le dataset, nous allons effectuer des étapes de préparation

- flake8 pour vérifier PEP8

In [ ]:
# %load_ext pycodestyle_magic
# %flake8_off

- imports des packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, metrics, preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import warnings
from pprint import pprint
# settings
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')

In [ ]:
# récupérer les données
raw = pd.read_csv('prep.csv')
print(raw.shape)

# 1. Appliquer (ou non) une restriction des données


In [ ]:
# appliquer la séléction "stricte"
sel = raw[~(((raw['LargestPropertyUseType'] == 'Multifamily Housing') & (
    (raw['SecondLargestPropertyUseType'] == 'None')) |
    (raw['SecondLargestPropertyUseType'] == 'Parking')) & (
    (raw['ThirdLargestPropertyUseType'] == 'None')))]
print(sel.shape)

In [ ]:
# ne pas appliquer de sélection 
sel = raw.copy()

In [ ]:
# features de résultats : 4 cibles possibles
Results = ['SiteEUI(kBtu/sf)',
           'SiteEnergyUse(kBtu)',
           'TotalGHGEmissions',
           'GHGEmissionsIntensity']

In [ ]:
# retirer les outliers
for i in Results:
    sel = sel[sel[i] <= sel[i].quantile(0.99)]
    sel = sel[sel[i] >= sel[i].quantile(0.01)]
    print(sel.shape)

In [ ]:
# compter le nombre de données manquantes pour l'ESS
sel['ENERGYSTARScore'].isna().sum()

- le test de prise en compte de l'ENERGYSTARScore impliquera d'opérer une restriction supplémentaire sur les données, donc jouer les modèles pour ajouter une référence avec et calculer l'apport de l'ENERGYSTARScore

In [ ]:
# opérer si nécessaire une sélection supplémentaire des données :
# entre 2015 et 2016, avec ou sans valeurs par défauts, avec ou sans outliers.
Select_features = ['OSEBuildingID',
                   'DataYear',
                   'Select_Default',
                   'Select_Outlier']

# 2.  Sélectionner la cible et des features

- pour rappel 3 catégories de features ont été crées : <br/>
Features_OH, suite One Hot Encoding des données catégorielles, <br/>
Features_RG, suite au traitement des données numériques, <br/>
et Features_UseType, suite au calcul de la proportion de type d'usage - qualitatif. <br/>
en plus des features dérivés (pseudo-numériques)<br/>
N_D_features = ['N_D_hasNaturalGas', 'N_D_hasSteam', 'N_D_NbPropUseRange', 'N_D_NbofBuildingsRange', 'N_D_NbofFloorsRange','N_D_BuildingType']

Le principe retenu est de : <br/>
- sélectionner les features à prendre en compte dans le modèle, avec Energystar score le cas échéant 
    - sel_features
- sélectioner la méthode de stratification<br/>
    - 'N_BuildingType' (une piste serait de tester aussi 'N_PrimaryPropertyType')
- jouer le modèle sur chacune des cibles parmi les 4 possibles (boucle for) <br/>
    - sel_target

# 3. Fonction d'évaluation des modèles

In [ ]:
# fonction d'évaluation d'un modèle
# entrainement, prédictions, calcul des scores et affichage predict vs test

def evaluate(model_name,
             model,
             data,
             sel_features,
             sel_target,
             split_size,
             stratif,
             imp):

    # sélection des features et de la cible
    features = data[sel_features]
    # passage au log de la cible
    labels = np.log(data[sel_target])

    # split
    train_features, test_features, train_labels, test_labels = train_test_split(
        features,
        labels,
        test_size=split_size,
        random_state=42,
        stratify=data[stratif])

    # entrainement du modèle
    model.fit(train_features, train_labels)

    # calcul des predictions
    predictions = model.predict(test_features)

    # retour aux valeurs d'origine
    test = np.exp(test_labels)
    predict = np.exp(predictions)

    # évaluation selon r2 et rmse
    r2 = r2_score(test, predict)
    rmse = np.sqrt(
        metrics.mean_squared_error(
            test,
            predict))
    mae = mean_absolute_error(test, predict)

    # affichage Test vs Predictions
    g = sns.jointplot(
        test,
        predict,
        kind='reg',
        stat_func=r2_score)
    g.ax_joint.set_xlabel(sel_target + ' - test')
    g.ax_joint.set_ylabel(sel_target + ' - predicted')
    g.fig.suptitle(t='Test vs Prediction for: '
                   + str(sel_target)
                   + '\n'
                   + 'Model: '
                   + str(model).split('(')[0]
                   + ' - '
                   + str(model_name),
                   y=0,
                   fontsize=16,
                   alpha=0.75,
                   weight='bold',
                   ha='center')
    x0, x1 = g.ax_joint.get_xlim()
    y0, y1 = g.ax_joint.get_ylim()
    lims = [max(x0, y0), min(x1, y1)]
    g.ax_joint.plot(lims, lims, ':k')
    plt.show()

    # affichage des scores et des paramètres du modèle
    print('r2: {:.2f}'.format(r2))
    print('rmse : {:.2f}'.format(rmse))
    print('mae : {:.2f}'.format(mae))
    print(str(model).split('(')[0])
    pprint(model.get_params())

    # affichage des erreurs
    df_res = pd.DataFrame({'true': test, 'pred': predict})
    df_res = df_res.sort_values('true')

    plt.plot(df_res['pred'].values, label='pred')
    plt.plot(df_res['true'].values, label='true')
    plt.xlabel('Test sample')
    plt.ylabel(sel_target)
    plt.suptitle(t='Error display for: '
                   + str(sel_target)
                   + '\n'
                   + 'Model: '
                   + str(model).split('(')[0]
                   + ' - '
                   + str(model_name),
                   y=0,
                   fontsize=16,
                   alpha=0.75,
                   weight='bold',
                   ha='center')
    plt.legend()
    plt.show()

    # affichage des features importance si option
    if imp == 1:
        feat_imp(model)

    # affichage de la valeur max prédite
    
        
    return r2, rmse, mae



In [ ]:
# fonction d'affichage des features importance d'un model randomforest

def feat_imp(rf_model):
    feature_importances = rf_model.feature_importances_
    temp = {
        'feature_names': sel_features,
        'feature_importances': feature_importances
    }
    temp = pd.DataFrame(temp).sort_values(by='feature_importances',
                                          ascending=False)
    display(temp.tail(30))

    scree = temp.feature_importances*100
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(), c="red", marker='o')
    plt.xlabel('features classés par importance décroissante')
    plt.ylabel('importance')
    plt.title('distribution des features importance')
    plt.show(block=False)



# 4. Définir le scope (features pris en compte et cible)

In [ ]:
# sélectionner les arguments de la fonction d'évaluation
# les features dérivés et les features numériques traités
sel_features = sel.columns[
    sel.columns.str.contains('N_RG_') |
    sel.columns.str.contains('N_D_')]
print(sel_features.size)
# la cible prise en référence est 
sel_target = 'SiteEnergyUse(kBtu)'

# 5. Modèles 

## 5.1. Dummy Regressor

In [ ]:
# créer une référence approche naïve par la moyenne
dummy_regr = DummyRegressor(strategy='median')

In [ ]:
# afficher les scores et prédictions vs test associées
for i in Results:
    dummy_score = evaluate('dummy_regr',
                           dummy_regr,
                           sel,
                           sel_features,
                           i,
                           0.25,
                           'N_D_BuildingType',
                           0)

## 5.2. Base Random Forest

In [ ]:
# créer une référence 
base_rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
# afficher les scores et prédictions vs test associées
for i in Results:
    base_rf_score = evaluate('base_rf', base_rf, sel, sel_features, i, 0.25, 'N_D_BuildingType', 1)

## 5.3. Randomized Search CV >> BEST MODEL <<

In [ ]:
# définition des plages de variation des hyperparamètres
n_estimators = [int(x) for x in np.linspace(start=300, stop=500, num=3)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(130, 190, num=7)]
max_depth.append(None)
min_samples_split = [2, 3]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

In [ ]:
# création de la grid : random_grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# sélection des features et de la cible (une seule cible)
features = sel[sel_features]
labels = np.log(sel[sel_target])

# split
train_features, test_features, train_labels, test_labels = train_test_split(
    features,
    labels,
    test_size=0.25,
    random_state=42,
    stratify=sel['N_D_BuildingType'])

In [ ]:
# créer d'un modèle et RandomizedSearchCV des hyperparamètres
# avec 3 folds de cross validation
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf,
                               param_distributions=random_grid,
                               n_iter=100,
                               cv=3,
                               verbose=2,
                               random_state=42,
                               n_jobs=-1)

In [ ]:
# entraîneer le modèle pour déterminer les meilleurs paramètres
rf_random.fit(train_features, train_labels)
rf_random.best_params_

In [ ]:
# appliquer les meilleurs paramètres
best_random = rf_random.best_estimator_
random_score = evaluate('best_random', best_random, sel, sel_features, sel_target, 0.25, 'N_D_BuildingType', 1)

## 5.4. Grid Search with Cross Validation

In [ ]:
# définir les paramètres pour un GridSearch CV
param_grid = {'bootstrap': [False],
              'max_depth': [140, 150, 160],
              'max_features': ['sqrt'],
              'min_samples_leaf': [1],
              'min_samples_split': [3],
              'n_estimators': [200, 300, 400]
              }

In [ ]:
# créer un model de base rf
rf = RandomForestRegressor()
# paramètrer le GridSearch, avec 3 folds
rf_grid_search = GridSearchCV(estimator=rf,
                              param_grid=param_grid,
                              cv=3,
                              n_jobs=-1,
                              verbose=2)

In [ ]:
# entraînement du modèle pour déterminer les meilleurs paramètres
rf_grid_search.fit(train_features, train_labels)
rf_grid_search.best_params_

In [ ]:
# application des meilleurs paramètres
rf_best_grid = rf_grid_search.best_estimator_
best_grid_score = evaluate('rf_best_grid', rf_best_grid, sel, sel_features, sel_target, 0.25, 'N_D_BuildingType', 1)

## 5.5. Linear Regression - base

In [ ]:
# créer un modèle de régression linéaire
base_lr = linear_model.LinearRegression()

In [ ]:
# afficher les scores et prédictions vs test associées
for i in Results:
    base_lr_score = evaluate('base_lr',
                             base_lr,
                             sel,
                             sel_features,
                             i,
                             0.25,
                             'N_D_BuildingType',
                             0)

## 5.6. Ridge avec Cross Validation

In [ ]:
# créer un modèle ridge with CV
ridgeCV = linear_model.RidgeCV(alphas=np.logspace(-1, 6, 200))

In [ ]:
# entrainer le ridge CV dans les mêms conditions que le random forest
model_cv = ridgeCV.fit(train_features, train_labels)
model_cv.alpha_

In [ ]:
best_ridgeCV = linear_model.Ridge(alpha=model_cv.alpha_)
best_ridgeCV_score = evaluate('best_ridgeCV', best_ridgeCV, sel, sel_features, sel_target, 0.25, 'N_D_BuildingType', 0)

## 5.7. Lasso avec Cross Validation

In [ ]:
# Appliquer le lasso
LassoCV = linear_model.LassoCV(alphas=np.logspace(-5, 2, 200), fit_intercept=False)


In [ ]:
# entrainer le ridge CV dans les mêms conditions que le random forest
LassoCv_mdl = LassoCV.fit(train_features, train_labels)
LassoCv_mdl.alpha_

In [ ]:
best_LassoCV = linear_model.Lasso(alpha=LassoCv_mdl.alpha_)
best_LassoCV_score = evaluate('best_LassoCV', best_LassoCV, sel, sel_features, sel_target, 0.25, 'N_D_BuildingType', 0)

# 6. Evaluation de l'intérêt de l'EnergyStar Score dans les prédiction d'émissions

- cette section compare les modèles avec ou sans prise en compte du feature EnergyStar Score. 
- l'apport est réel mais pas flagrant 5 % d'amélioration du R2, et d'autant moindre qu'il y a de feature (et feature engineering) pris en compte.

In [ ]:
# retourner aux données initiales
sel = raw.copy()

In [ ]:
# features de résultats : 4 cibles possibles
Results = ['SiteEUI(kBtu/sf)',
           'SiteEnergyUse(kBtu)',
           'TotalGHGEmissions',
           'GHGEmissionsIntensity']

In [ ]:
# retirer les outliers
for i in Results:
    sel = sel[sel[i] <= sel[i].quantile(0.99)]
    sel = sel[sel[i] >= sel[i].quantile(0.01)]
    print(sel.shape)

In [ ]:
# compter le nombre de données manquantes pour l'ESS
sel['ENERGYSTARScore'].isna().sum()

In [ ]:
sel.dropna(axis=0, subset=['ENERGYSTARScore'], inplace=True)
sel.shape

In [ ]:
# sélectionner les arguments de la fonction d'évaluation
# les features dérivés et les features numériques traités
sel_features = sel.columns[
    sel.columns.str.contains('N_RG_') |
    sel.columns.str.contains('N_D_') |
    sel.columns.str.contains('N_OH_')]
print(sel_features.size)
# la cible prise en référence est 
sel_target = 'TotalGHGEmissions'

In [ ]:
# évaluer un randomforest sur ce scope sans l'ESS
base_rf_woESS = RandomForestRegressor(n_estimators=100, random_state=42)
base_rf_woESS_score = evaluate('base_rf_woESS',
                               base_rf_woESS,
                               sel,
                               sel_features,
                               'GHGEmissionsIntensity',
                               0.25,
                               'N_D_BuildingType',
                               1)

In [ ]:
# ajouter l'EnergyStar Score
sel_features = sel_features.append(sel.columns.intersection(['ENERGYSTARScore']))

In [ ]:
# évaluer un randomforest sur ce scope avec ESS
base_rf_wiESS = RandomForestRegressor(n_estimators=100, random_state=42)
base_rf_wiESS_score = evaluate('base_rf_wiESS',
                               base_rf_wiESS,
                               sel,
                               sel_features,
                               'GHGEmissionsIntensity',
                               0.25,
                               'N_D_BuildingType',
                               1)

'- le gain avec l'ESS n'est pas vraiment significatif sur cette base, on évalue également ce que ce feature peut apporter dans le cas d'un Randomized Search with CV

In [ ]:
# définition des plages de variation des hyperparamètres
n_estimators = [int(x) for x in np.linspace(start=300, stop=500, num=3)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(130, 190, num=7)]
max_depth.append(None)
min_samples_split = [2, 3]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

In [ ]:
# création de la grid : random_grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# sélectionner les arguments de la fonction d'évaluation
# les features dérivés et les features numériques traités
sel_features = sel.columns[
    sel.columns.str.contains('N_RG_') |
    sel.columns.str.contains('N_D_') |
    sel.columns.str.contains('N_OH_')]
print(sel_features.size)
# la cible prise en référence est 
sel_target = 'TotalGHGEmissions'

In [ ]:
# sélection des features et de la cible (une seule cible)
features = sel[sel_features]
labels = np.log(sel[sel_target])

# split
train_features, test_features, train_labels, test_labels = train_test_split(
    features,
    labels,
    test_size=0.25,
    random_state=42,
    stratify=sel['N_D_BuildingType'])

In [ ]:
# créer d'un modèle et RandomizedSearchCV des hyperparamètres
# avec 3 folds de cross validation
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf,
                               param_distributions=random_grid,
                               n_iter=100,
                               cv=3,
                               verbose=2,
                               random_state=42,
                               n_jobs=-1)

In [ ]:
# entraîner le modèle pour déterminer les meilleurs paramètres
rf_random.fit(train_features, train_labels)
rf_random.best_params_

In [ ]:
# appliquer les meilleurs paramètres
best_random = rf_random.best_estimator_
random_score = evaluate('best_random', best_random, sel, sel_features, sel_target, 0.25, 'N_D_BuildingType', 1)

# 7. Gradient Boosting regressor

In [ ]:
# sélectionner les arguments de la fonction d'évaluation
# les features dérivés et les features numériques traités
sel_features = sel.columns[
    sel.columns.str.contains('N_RG_') |
    sel.columns.str.contains('N_D_') |
    sel.columns.str.contains('N_OH_')]
print(sel_features.size)
# la cible prise en référence est 
sel_target = 'SiteEnergyUse(kBtu)'

In [ ]:
# créer un modèle GBM
GBM_base = GradientBoostingRegressor(n_estimators=300,
                                     max_depth=150,
                                     criterion='mae',
                                     learning_rate=0.1,
                                     random_state=42)

In [ ]:
# afficher les scores et prédictions vs test associées
base_rf_score = evaluate('GBM_base', GBM_base, sel, sel_features, 'SiteEnergyUse(kBtu)', 0.25, 'N_D_BuildingType', 0)

# 8. xGBoost regressor


In [ ]:
# créer un modèle xGboost
xgb_base = xgb.XGBModel()

In [ ]:
xgb_base.get_params

In [ ]:
xgb_base = xgb.XGBModel(max_depth=150, n_estimators=300, objective='reg:squarederror')

In [ ]:
# afficher les scores et prédictions vs test associées
xgb_score = evaluate('xgb_base', xgb_base, sel, sel_features, 'SiteEnergyUse(kBtu)', 0.25, 'N_D_BuildingType', 1)

# 9. Validation

## 9.1 Confronter le modèle retenu à des données test

In [ ]:
# isoler des bâtiments pour validation
buldings_in15not16 = pd.read_csv('in15not16.csv')
buldings_in16not15 = pd.read_csv('in16not15.csv')

In [ ]:
sel = raw.copy()

In [ ]:
# features de résultats : 4 cibles possibles
Results = ['SiteEUI(kBtu/sf)',
           'SiteEnergyUse(kBtu)',
           'TotalGHGEmissions',
           'GHGEmissionsIntensity']

In [ ]:
# retirer les outliers
for i in Results:
    sel = sel[sel[i] <= sel[i].quantile(0.99)]
    sel = sel[sel[i] >= sel[i].quantile(0.01)]
    print(sel.shape)

In [ ]:
# créer un jeu de données validation
valid1 = sel[sel['OSEBuildingID'].isin(buldings_in16not15['OSEBuildingID'])]
valid2 = sel[sel['OSEBuildingID'].isin(buldings_in15not16['OSEBuildingID'])] 

In [ ]:
# concaténer les données 
valid = pd.concat([valid1, valid2], sort=False).reset_index(drop=True)

In [ ]:
sel = sel[~sel['OSEBuildingID'].isin(buldings_in16not15['OSEBuildingID'])]
sel = sel[~sel['OSEBuildingID'].isin(buldings_in16not15['OSEBuildingID'])]

In [ ]:
sel.shape

In [ ]:
valid.shape

le modèle retenu est le random forest avec une sélection "sobre" de features, on fit de nouveau ce modèle, mais hors données de test. 

## 9.2 Appliquer le Best Model  - Randomized Search CV

In [ ]:
# sélectionner les arguments de la fonction d'évaluation
# les features dérivés et les features numériques traités
sel_features = sel.columns[
    sel.columns.str.contains('N_RG_') |
    sel.columns.str.contains('N_D_')]
print(sel_features.size)
# la cible prise en référence est 
sel_target = 'SiteEnergyUse(kBtu)'

In [ ]:
# définition des plages de variation des hyperparamètres
n_estimators = [int(x) for x in np.linspace(start=300, stop=500, num=3)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(130, 190, num=7)]
max_depth.append(None)
min_samples_split = [2, 3]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

In [ ]:
# création de la grid : random_grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# sélection des features et de la cible (une seule cible)
features = sel[sel_features]
labels = np.log(sel[sel_target])

# split
train_features, test_features, train_labels, test_labels = train_test_split(
    features,
    labels,
    test_size=0.25,
    random_state=42,
    stratify=sel['N_D_BuildingType'])

In [ ]:
# créer d'un modèle et RandomizedSearchCV des hyperparamètres
# avec 3 folds de cross validation
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf,
                               param_distributions=random_grid,
                               n_iter=100,
                               cv=3,
                               verbose=2,
                               random_state=42,
                               n_jobs=-1)

In [ ]:
# entraîner le modèle pour déterminer les meilleurs paramètres
rf_random.fit(train_features, train_labels)
rf_random.best_params_

In [ ]:
# appliquer les meilleurs paramètres
best_random = rf_random.best_estimator_
random_score = evaluate('best_random', best_random, sel, sel_features, sel_target, 0.25, 'N_D_BuildingType', 1)

- excellente surprise au passage d'observer un meilleur score pour ce découpage spécifique

In [ ]:
# sélection des features et de la cible
features = valid[sel_features]
# passage au log de la cible
labels = np.log(valid[sel_target])
# calcul des predictions
predictions = best_random.predict(features)
# retour aux valeurs d'origine
test = np.exp(labels)
predict = np.exp(predictions)
# évaluation selon r2 et rmse
r2 = r2_score(test, predict)
# affichage du score r2
print('r2: {:.2f}'.format(r2))

In [ ]:
# affichage Test vs Predictions
g = sns.jointplot(
    test,
    predict,
    kind='reg',
    stat_func=r2_score)
g.ax_joint.set_xlabel(sel_target + ' - valid')
g.ax_joint.set_ylabel(sel_target + ' - predicted')
g.fig.suptitle(t='valid vs Prediction : '
                   + str(sel_target),
                   y=0,
                   fontsize=16,
                   alpha=0.75,
                   weight='bold',
                   ha='center')
x0, x1 = g.ax_joint.get_xlim()
y0, y1 = g.ax_joint.get_ylim()
lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, ':k')
plt.show()